In [2]:
# import all the necessary libs (navigation is a local .py file with helper functions)

import navigation as nav
import pandas as pd
import glob
import zipfile
from unidecode import unidecode

In [3]:
# define path of .zip files, add the path names to a list

path = "other_datasets\\ptransp_viagens\\*"
files = glob.glob(path)
df = pd.DataFrame()

In [ ]:
# open the files and add them to a pandas dataframe, then add all to df (final)

for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        datasets = zip_ref.namelist()
        with zip_ref.open(datasets[-1]) as my_file:  # important, for we only want the last .csv withing each .zip
            df1 = pd.read_csv(my_file, encoding='latin1', sep=';', dtype='object')
            df = pd.concat([df, df1], ignore_index=True)
            print(f'{file}: ok!')

In [5]:
# create a list to rename the columns (use print(df.info()) to see the current col names if needed

col_remap = [
    'id_viagem',                # int
    'num_proposta',             # int
    'situacao',                 # obj
    'viagem_urgente',           # bool
    'justificativa_urgencia',   # obj
    'cod_orgao_superior',       # int
    'nome_orgao_superior',      # obj
    'cod_orgao_solicitante',    # int
    'nome_orgao_solicitante',   # obj
    'cpf_viajante',             # obj
    'nome_viajante',            # obj
    'cargo',                    # obj
    'funcao',                   # obj
    'descricao_funcao',         # obj
    'dt_inicio',                # datetime
    'dt_fim',                   # datetime
    'destinos',                 # obj
    'motivo',                   # obj
    'valor_diarias',            # float
    'valor_passagens',          # float
    'valor_devolucao',          # float
    'valor_outros_gastos'       # float
]

df.columns = col_remap

In [ ]:
print(df.info())

In [7]:
# # DATA TREATMENT

# normalize objects: all uppercase, no special characters
treated_colums1 = [
    'situacao',                 # obj
    'justificativa_urgencia',   # obj
    'nome_orgao_superior',      # obj
    'nome_orgao_solicitante',   # obj
    'cpf_viajante',             # obj
    'nome_viajante',            # obj
    'cargo',                    # obj
    'funcao',                   # obj
    'descricao_funcao',         # obj
    'destinos',                 # obj
    'motivo'                    # obj
]

def obj_normalizer(row):
    if pd.isna(row):
        return row
    try:
        normalized = unidecode(row).upper()
        return normalized
    except Exception as e:
        print(f'Error occured: {e}')
        return row

df[treated_colums1] = df[treated_colums1].map(obj_normalizer)

In [8]:
# transforming from obj to datetime

treated_colums2 = [
    'dt_inicio',                # datetime
    'dt_fim',                   # datetime
]

def date_treatment(row):
    try:
        return pd.to_datetime(row, dayfirst=True)
    except ValueError as e:
        print(f'Error on {row}: {e}')
        return row
df[treated_colums2] = df[treated_colums2].map(date_treatment)

In [9]:
# transforming from obj to Int64

treated_colums4 = [
    'id_viagem',
    'cod_orgao_superior',
    'cod_orgao_solicitante'
]

for col in treated_colums4:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(-1).astype('Int64')


In [10]:
# transforming from obj to float

treated_colums3 = [
    'valor_diarias',            # float
    'valor_passagens',          # float
    'valor_devolucao',          # float
    'valor_outros_gastos'       # float
]

for col in treated_colums3:
    df[col] = df[col].str.replace(',', '.').astype(float)

In [11]:
# transforming from obj to bool

df['viagem_urgente'] = df['viagem_urgente'].map({'SIM': 1, 'NÃO': 0})
df['viagem_urgente'] = df['viagem_urgente'].astype(bool)

In [ ]:
# Check if properly treated

print(df.head(5))
print(df.info())

In [13]:
def save_to_sql_in_chunks(df, table_name, conn, chunksize=100000):
    """
    Save a large DataFrame to a SQL table in chunks.

    Parameters:
    - df: pandas.DataFrame
        The DataFrame to be saved.
    - table_name: str
        The name of the table in the database.
    - conn: SQLAlchemy or sqlite3 connection
        The database connection.
    - chunksize: int, optional (default=100000)
        The number of rows per chunk to write to the database.

    Returns:
    - None
    """
    from sqlalchemy.exc import SQLAlchemyError

    try:
        for i in range(0, len(df), chunksize):
            chunk = df.iloc[i:i+chunksize]
            # Append each chunk to the table
            chunk.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Chunk {i // chunksize + 1} written to table {table_name}")
    except SQLAlchemyError as e:
        print(f"An error occurred: {e}")

In [ ]:
# send to sqlite3 database

save_to_sql_in_chunks(df, 'ptransp.viagens', nav.sqlite_conn)